In [2]:
from pyspark import SparkContext, SparkConf
import re

1. Напишите программу, которая находит самое длинное слово.

In [7]:
sc.stop()
conf = SparkConf().setAppName("LongestWord").setMaster("local")
sc = SparkContext(conf=conf)

data = sc.textFile("D:/big-data-2023/alms-bd-2023-3cecad0d39f8/data/wiki.txt")

longest_word = (
    data.flatMap(lambda line: line.replace('\t', ' ').split(" "))
    .reduce(lambda x, y: x if len(x) > len(y) and x.find('http') == -1 and x.find('.') == -1 else y)
)

print("Longest word:", longest_word)


Longest word: землемерно-таксаторские


2. Напишите программу, которая находит среднюю длину слов.

In [19]:
avg_word_length = (
    data.flatMap(lambda line: line.split(" "))
    .map(lambda word: (len(word), 1))
    .reduce(lambda x, y: (x[0] + y[0], x[1] + y[1]))
)

avg_word_length = avg_word_length[0] / avg_word_length[1]
print("Average word length:", avg_word_length)

Average word length: 6.582369447100479


3. Напишите программу, которая находит самое частоупотребляемое слово, состоящее из латинских букв.

In [22]:
words = (
    data.flatMap(lambda line: re.findall(r'\b[a-zA-Z]+\b', line))
    .map(lambda word: word.lower())
)

most_frequent_word = words.map(lambda word: (word, 1)).reduceByKey(lambda x, y: x + y).max(lambda x: x[1])[0]

print("Most frequent word:", most_frequent_word)

Most frequent word: ru


4. Все слова, которые более чем в половине случаев начинаются с большой буквы и встречаются больше 10 раз.

In [38]:
words = data.flatMap(lambda line: line.split(" "))
# считаем количество вхождений каждого слова
capitalized_words = words.filter(lambda word: len(word) > 0 and word[0].isupper())

word_counts = capitalized_words.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)

# находим общее количество слов
total_words = capitalized_words.count()

# фильтруем слова, которые встречаются больше 10 раз и начинаются с большой буквы в более, чем половине случаев
result = word_counts.filter(lambda pair: pair[1] > 10 and (pair[0][0].isupper() and pair[1] / total_words > 0.5)).collect()

# выводим результат
for word, count in result:
    print(word)

5. Напишите программу, которая с помощью статистики определяет устойчивые сокращения вида `пр.`, `др.`, ...

In [84]:
sc.stop()
conf = SparkConf().setAppName("Stable Abbreviations")
sc = SparkContext(conf=conf)

# загружаем текстовый файл и разбиваем его на слова
words = sc.textFile("D:/big-data-2023/alms-bd-2023-3cecad0d39f8/data/wiki.txt").flatMap(lambda line: line.split(" "))

# фильтруем слова, чтобы оставить только те, которые содержат сокращения
abbreviations = (words.flatMap(lambda line: re.findall(r'\b[а-яА-Я]+\.\b', line)).filter(lambda word: len(word) == 3 and word.find('.') != -1 and word[:-1].isalpha()))

# считаем количество вхождений каждого сокращения
abbreviation_counts = abbreviations.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)

# считаем общее количество слов
total_words = words.count()

# фильтруем сокращения, которые встречаются более 10 раз и составляют более 0.1% от общего количества слов
result = abbreviation_counts.filter(lambda pair: pair[1] > 10 and pair[1] / total_words > 0.0000001).collect()

# выводим результат
for abbreviation, count in result:
    print(abbreviation)


ед.
кв.
мн.
ст.
ок.
см.
фр.


6. Напишите программу, которая с помощью статистики определяет устойчивые сокращения вида  `т.п.`, `н.э.`, ...

In [96]:
sc.stop()
conf = SparkConf().setAppName("Stable Abbreviations")
sc = SparkContext(conf=conf)

# загружаем текстовый файл и разбиваем его на слова
words = sc.textFile("D:/big-data-2023/alms-bd-2023-3cecad0d39f8/data/wiki.txt").flatMap(lambda line: line.split(" "))

# фильтруем слова, чтобы оставить только те, которые содержат сокращения
abbreviations = (words.flatMap(lambda line: re.findall(r'\b[а-яА-Я]+\.+\b[а-яА-Я]\b', line)).filter(lambda word: word.find('.') != -1 and word.find('.') != (len(word)-1)))

# считаем количество вхождений каждого сокращения
abbreviation_counts = abbreviations.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)

# считаем общее количество слов
total_words = words.count()

# фильтруем сокращения, которые встречаются более 10 раз и составляют более 0.1% от общего количества слов
result = abbreviation_counts.filter(lambda pair: pair[1] > 10 and pair[1] / total_words > 0.000001).collect()

# выводим результат
for abbreviation, count in result:
    print(abbreviation)


т.д
г.г
т.к
т.н
л.с
с.ш
в.д
А.А
мн.ч
т.е
в.в
н.э
ю.ш
г.р
ед.ч
В.В
т.ч
э.д
кв.м
А.И
т.п
В.И
А.Ф
